# Stimmungsanalyse mit dem DB-Social-Media Datensatz

Der Datensatz wurde zur Stimmungsanalyse und Erkennung von relevanten Feedback erstellt.
Weitere Dokumentation und Downloads sind unter https://sites.google.com/view/germeval2017-absa/data?authuser=0 zu finden.


Der Datensatz wird mittels Logistischer Regression, FastText und Bert bearbeitet.
# Jupyter-Notebook

Jupyter-Notebooks sind interaktive Python-Scripte, in denen Markdown und sogar Latex zur Dokumentation verwendet werden kann. In diesem Abschnitt sollen einerseits allgemeine Informationen wie Tastenkombinationen und übliche Workflows, andererseits aber auch für diesen Workshop und die verwendete Hardware spezifische Informationen im Umgang mit Python und Jupyter-Notebooks geliefert werden.

### Wichtiges auf einen Blick (a.k.a. TL;DR)

- **beim Wechseln auf ein anderes Notebook immer den Kernel beenden (Kernel -> Shutdown)**
- **Shift + Enter** zum Ausführen der aktiven Zelle
- **Strg + Shift + p** zum Öffnen der Kommandopalette
- **Shift + o** zum Togglen des Zell-Scrollings
- bei Fehlermeldungen im Zweifel **Kernel neustarten (Kernel -> Restart)**
- "Hilfe, ich sehe den Markdown-Code" -> **Shift + Enter** in der entsprechenden Zelle
- "Hilfe, ich bekommen ResourceExhaustion / OutOfMemory (OOM) Fehler" -> **Kernel neustarten, Kernel von anderen noch laufenden Notebooks herunterfahren** (oben links auf das Jupyter-Logo klicken, dann auf den Tab "Running")
- "Hilfe, mein Notebook ist kaputt" -> siehe **Notebook "Wiederherstellung"**
- "Passiert da noch was?" -> ist der **Kreis oben rechts neben "Python 3", ausgefüllt und dunkel** dann ist der Kernel noch am Arbeiten, ist er nicht gefüllt dann ist der Kernel untätig. Die aktuell laufende Zelle ist die von oben gesehen erste bei der auf der linken Seite "In[*]" anstelle von z.B. "In[5]" steht. Es kann aber passieren dass sich der Kernel aufhängt, in dem Fall einfach oben auf __Kernel -> Interrupt__ und die Zelle erneut ausführen


### Überblick & Workflow

Die einzigen beiden Shortcuts die man sich eigentlich nur merken muss sind

- **Shift + Enter** zum Ausführen einer Zelle, und
- **Strg + Shift + p** zum Öffnen der Kommandopalette, von der aus man dann direkt Zugriff auf alle möglichen Befehle hat, inklusive entsprechender Shortcuts

Ein weiterer nützlicher Shortcut ist **Shift + o**, welcher das **Scrolling für Zellenoutput** umschaltet.

Zum **Editieren einer Zelle** genügt ein Doppelklick in die Zelle, bei Code-Zellen reicht es zum Beenden des Editiermodus einfach außerhalb der Zelle zu klicken, bei Dokumentationszellen (wie dieser hier) ist eine Ausführung der Zelle nötig um die Code-Ansicht zu verlassen.

Während eine Zelle ausgeführt wird, wechselt der Kernel-Indikator oben rechts neben "Python 3" von einem hellen Kreis mit dunklem Rand zu einem ausgefüllten dunklen Kreis und springt wieder zurück sobald die Ausführung beendet ist. Wenn mehrere Zellen gleichzeitig ausgeführt wurden, kann man an dem Label in der linken Spalte ablesen, ob die Zelle fertig ausgeführt wurde (**In [ZAHL]:**) oder ob sie gerade ausgeführt wird bzw. auf Ausführung wartet (**In [*]:**). Zusätzlich wird nach Ausführung einer Zelle die Zellenausgabe unterhalb der Zelle angezeigt.

Um den Überblick zu behalten kann es manchmal sinnvoll sein, die **Zellenausgabe zu löschen**. Dies kann u.a. auf diesen beiden Wegen erfolgen:

- oben auf Cell -> Current Outputs / All outputs -> Clear
- oben auf Kernel -> Restart & Clear Output

Der Kernel ist für die Ausführung des Python-Codes zuständig und behält den Kontext (also belegte Variablen, definierte Funktionen und belegter Speicher) seit dem letzten Kernel-(Neu)start. Dies kann zu Problemen führen wenn Zellen in anderer Reihenfolge ausgeführt werden oder Zellen übersprungen werden in denen Variablen oder Funktionen definiert werden die im weiteren Verlauf des Scripts benötigt werden, aber auch wenn **ein anderes Jupyter-Notebook gestartet wird**, da dafür ein weiterer Kernel gestartet wird.

Deswegen beim **Wechseln auf ein anderes Notebook** immer den **Kernel herunterfahren oder neustarten** (oben Kernel -> Restart/Shutdown).

In [ ]:
import os
import pandas as pd
import re
import numpy as np
import gensim
import sys
sys.path.insert(1, '/data/db_twitter/')
import data_cleaner
import fasttext
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import random
import time
import datetime

import sys
import random as rn
import torch
from transformers import BertModel, BertTokenizer
from transformers import BertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelWithLMHead
import transformers as ppb
import csv
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from apex import amp

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn import linear_model

ROOT_DIR = os.path.abspath("/workspace")
DATA_DIR = os.path.abspath("/data")

TRAIN_DF=DATA_DIR+"/db_twitter/train_v1.4.tsv"
TEST_DF=DATA_DIR+"/db_twitter/dev_v1.4.tsv"
CLEAN_TRAIN_DF=DATA_DIR+"/db_twitter/train_clean.tsv"
CLEAN_TEST_DF=DATA_DIR+"/db_twitter/test_clean.tsv"


# 1. Datenanalyse
Zur Analyse laden wir den Testdatensatz ein und schauen uns die erste Zeile an.

In [ ]:
analyse_train = pd.read_csv(TRAIN_DF, delimiter='\t', header=None )
analyse_test = pd.read_csv(TEST_DF, delimiter='\t', header=None )

print("Trainingsmenge:")
print(analyse_train[:5])
print("\n"+analyse_train[1].iloc[0])

print("\n\nTestmenge:")
print(analyse_test[:2])
print("\n"+analyse_test[1].iloc[0])


## 1.1 Klassenverteilung
Die Verteilung der Klassen sieht wie folgt aus:

In [ ]:
unique_train, counts_train = np.unique(analyse_train[3], return_counts=True)
unique_test, counts_test = np.unique(analyse_test[3], return_counts=True)

print("Trainingsmenge: ")
print((np.asarray((unique_train, counts_train)).T))

print("\nTestmenge: ")
print(np.asarray((unique_test, counts_test)).T)

# 2. Daten bereinigen

Für unsere Aufgabe benötigen wir nur den Text der Posts und die Labels, daher nutzen wir nur die Spalten 1 und 3.

Um Erwähnungen zu vereinfachen, wird jeder Twitterusername durch den Text 'twitterusername' ersetzt. @DB_Bahn wird besonders gekennzeichnet, da er hier von besonderer Bedeutung ist.

Zudem werden Satzzeichen entfernt und Emojis durch festgelegte Wörter ersetzt.

In [ ]:
train_data=pd.read_csv(TRAIN_DF, delimiter='\t', usecols=[1,3], header=None )
test_data=pd.read_csv(TEST_DF, delimiter='\t', usecols=[1,3], header=None )

print(train_data[:5])

In [ ]:
print("Datensätze werden bereinigt, dies kann einige Zeit in anspruch nehmen...")
clean_train_data=train_data.apply(data_cleaner.clean_text)
clean_test_data=test_data.apply(data_cleaner.clean_text)
print("Datensätze bereinigt.")

print(clean_train_data[:5])

Die bereinigten Datensätze sind bereits gespeichert und können im Folgenden geladen werden.
Ein Vergleich der originalen und bereinigten Daten kann hier angesehen werden. Über das Argument in `.iloc[]`kann der Eintrag gewählt werden. 

In [ ]:
print("original: ",train_data[1].iloc[2])
print("\nbereinigt: ",clean_train_data[1].iloc[2])

## Bereinigte Trainings und Testdaten laden
**Wird verwendet, falls das Bereinigen übersprungen werden soll**

In [ ]:
clean_train_data = pd.read_csv(CLEAN_TRAIN_DF, delimiter='\t',header=None)
clean_test_data = pd.read_csv(CLEAN_TEST_DF, delimiter='\t',header=None)

# 3. Logistische-regression mit Gensim

Der Trainings und Testdatensatz wird mit Gensim zum vectorisieren vorverarbeitet:

In [ ]:
def read_corpus(fname):
    for i in range(0,fname.shape[0]):
            tokens = gensim.utils.simple_preprocess(str(fname[0].iloc[i]))
            yield gensim.models.doc2vec.TaggedDocument(tokens, str(fname[1].iloc[i]))
                
train_corpus = list(read_corpus(clean_train_data))
test_corpus = list(read_corpus(clean_test_data))       

print("Trainingsdaten:")
print(train_corpus[:1])
print("\nTestdaten:")
print(test_corpus[:1])

## 3.1 Modell erzeugen
Gensim wird für ganze Sätze konfiguriert(docs) und das Wörterbuch wird erstellt. "min_count" ignoriert alle Wörter, die seltener als definiert vorkommen.

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=15)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Zunächst werden Text und Labels für einfache Weiterverwendung extrahiert, danach werden die Trainingsdaten von dem Modell vorhergesagt und zwischengespeichert.

In [ ]:
train_targets, train_regressors = zip(*[(doc.words, doc.tags) for doc in train_corpus])
test_targets, test_regressors = zip(*[(doc.words, doc.tags) for doc in test_corpus])

In [ ]:
X = []
for i in range(len(train_targets)):
    X.append(model.infer_vector(train_targets[i]))
train_x = np.asarray(X)
train_x.shape

## 3.2 Logistische Regression
Die erwarteten Labels werden an die Logistische regression gegeben

In [ ]:
Y = np.asarray(train_regressors)
le = preprocessing.LabelEncoder()
le.fit(Y)
train_y = le.transform(Y)

logreg = linear_model.LogisticRegression()
logreg.fit(train_x, train_y)

## 3.3 Genauigkeit der Testmenge
Hier wird die Testmenge von dem Modell analysiert. Die Zusammenfassung wird von SKLearn angezeigt:

In [ ]:
test_list = []
for i in range(len(test_targets)):
    test_list.append(model.infer_vector(test_targets[i]))
test_x = np.asarray(test_list)


test_Y = np.asarray(test_regressors)
test_y = le.transform(test_Y)

preds = logreg.predict(test_x)

print(classification_report(test_y,preds,target_names=["negative","neutral", "positive"]))

# 4 FastText

[FastText](https://fasttext.cc/) ist ein Framework von Facebook zum Klassifizieren von Texten. Es ist effizient und kann sogar auf Mobilgeräten verwendet werden.


## 4.1 Vorverarbeitung für FastText
Zuerst werden die Kommentare mit Labels versehen. Sie werden aus dem Datensatz in diese Form gebracht: <br>
"\__label\__<positive/neutral/negative> >Kommentar<".
    
Die ersten zwei Einträge werden angezeigt.

In [ ]:
def addLabels(dataframe):
    for i in range(0,dataframe.shape[0]):
        yield "__label__"+str(dataframe[1].iloc[i])+" "+ str(dataframe[0].iloc[i])
        
        
labeledList_train=list(addLabels(clean_train_data))
labeledList_test=list(addLabels(clean_test_data))

with open(ROOT_DIR+"/fasttext_train.txt",'w') as f:
    for line in labeledList_train:
        f.write(line+"\n")

with open(ROOT_DIR+"/fasttext_test.txt",'w') as f:
    for line in labeledList_test:
        f.write(line+"\n")

print(labeledList_train[:2])

## 4.2 Training
Anschließend wird mit FastText der Text trainiert. Die Anzahl der Epochen, Lernrate ([0.1,1]) und NGram-länge ([1,5]) kann angepasst werden.

In [ ]:
model = fasttext.train_supervised(input=ROOT_DIR+"/fasttext_train.txt",epoch=20,lr=0.2,wordNgrams=4)

## 4.3 Test des Modells
Um das Modell zu testen, wird der Testdatensatz übergeben.

Die Ausgabe ist wie folgt Formatiert: (Samples, Precision, Recall)

In [ ]:
model.test(ROOT_DIR+"/fasttext_test.txt")

## 4.4 Einzelinferenz
Hier können eigene Sätze mit dem Modell getestet werden

In [ ]:
to_predict="@DB_Bahn Heute mal wieder viel zu spät :("

#preprocess input
df=pd.DataFrame([to_predict])
df=df.apply(data_cleaner.clean_text)
to_predict=(df[0].item())

print("'",to_predict,"'")

model.predict(to_predict, k=-1, threshold=0.3)


# 5 Bert
[Bert](https://huggingface.co/transformers/model_doc/bert.html) (Bidirectional Encoder Representations from Transformers) ist ein Modell, das nicht nur die Wörter in einem Text, sondern auch den Kontext analysiert.

In diesem Notebook wird Pytorch als Backend verwendet.

In [ ]:
clean_train_data = pd.read_csv(CLEAN_TRAIN_DF, delimiter='\t',header=None)
clean_test_data = pd.read_csv(CLEAN_TEST_DF, delimiter='\t',header=None)
print(clean_train_data[:2])

## 5.1 Preprocessing
Das Modell erwartet die Labels als Ordinale, daher werden sie umgewandelt.

In [ ]:
def toOrdinal(n):
    if n=='positive':
        return 2
    if n=='neutral':
        return 1
    if n=='negative':
        return 0
        
bert_train=pd.DataFrame({'label':clean_train_data[1].apply(toOrdinal),
                         'text':clean_train_data[0]})

bert_test=pd.DataFrame({'label':clean_test_data[1].apply(toOrdinal),
                         'text':clean_test_data[0]})

Der Text wird mit Hilfe von BERT in Tokens mit Start- und Stopp-Symbolen umgewandelt. Es werden nur Eingaben von 512 Zeichen unterstützt.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased", do_lower_case=True)
print("Token werden erstellt, dies kann einige Zeit in anspruch nehmen..")
train_tokenized = bert_train['text'].astype(str).apply((lambda x: tokenizer.encode(x[:512], add_special_tokens=True)))
test_tokenized = bert_test['text'].astype(str).apply((lambda x: tokenizer.encode(x[:512], add_special_tokens=True)))
print("Token erstellt!")

print("\nText:",clean_train_data[0].iloc[0])
print("\nAls Token:",train_tokenized[0])

## 5.2 Padding der Daten
Da auf die Daten möglichst performant zugegriffen werden soll, werden sie als Arrays gespeichert. Die längste Zeichenkette gibt daher die größe der Arrays vor. Alle anderen werden am ende mit 0 aufgefült. Die `Attention Mask` filtert die aufgefüllten Werte später raus.

In [ ]:
max_len = 0
for i in train_tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        
for i in test_tokenized.values:
    if len(i) > max_len:
        max_len = len(i)        

            

train_padded = np.array([i + [0]*(max_len-len(i)) for i in train_tokenized.values])
train_attention_mask = np.where(train_padded != 0, 1, 0)

test_padded = np.array([i + [0]*(max_len-len(i)) for i in test_tokenized.values])
test_attention_mask = np.where(test_padded != 0, 1, 0)

## 5.3 Daten für GPU vorbereiten
Hier werden Datensätze zum Training und Testen erzeugt. Übergeben werden die Token-Sätze, Masken und Label. Die `Dataloader` laden die Datensätze mit der Größe `BATCH_SIZE` wenn benötigt auf die GPU, um den GPU-Speicher nicht durch den Datensatz auszulasten. In diesem Fall werden gleichzeitig 50 Einträge verarbeitet.

In [ ]:
train_dataset= TensorDataset(torch.tensor(train_padded),
                             torch.tensor(train_attention_mask),
                             torch.tensor(list(bert_train['label'])))

test_dataset= TensorDataset(torch.tensor(test_padded),
                            torch.tensor(test_attention_mask),
                            torch.tensor(list(bert_test['label'])))

BATCH_SIZE=50
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), 
            batch_size = BATCH_SIZE
        )

validation_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            batch_size = BATCH_SIZE
        )

## 5.4 Modell erstellen
Es wird ein BERT Modell mit 12 Layern erzeugt, die Anzahl an Labels muss mit dem Datensatz übereinstimmen (hier "positive", "neutral", "negative").

Der "cuda()" Aufruf verschiebt das Modell auf die GPU

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-german-cased",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)
model.cuda()

## 5.5 Optimizer und Lernrate
Als Optimizer verwenden wir Adam, da dieser sehr schnell konvergiert.
Um möglichst viele Daten gleichzeitig zu verarbeiten, verwenden wir `half-precision`. Wenn möglich, werden Float16 anstelle von Float32 Werte verwendet, wodurch Ressourcen gespart werden. Die Level werden von [APEX](https://nvidia.github.io/apex/amp.html) definiert und können nachgeschlagen werden.

In [ ]:
optimizer = Adam(model.parameters(),
                  lr = 0.000003, # args.learning_rate - default is 5e-5,
                  eps = 0.0000001 # args.adam_epsilon  - default is 1e-8.
                )
model, optimizer = amp.initialize(model, optimizer,opt_level="O1")

## 5.5 Training vorbereiten
Die Anzahl an Epochen kann über `epochs` eingestellt werden. Die Lernrate wird über einen scheduler gesteuert.
Die Funktionen dienen zur darstellung des Fortschritts im Training

In [ ]:
epochs = 15

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


## 5.6 Training
Das Training bestet aus dem Training mit der Trainingsmenge und Validierung mit der Validierungsmenge.

In [ ]:
# use cuda device
device = torch.device("cuda")

# same seed for reprodiction
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


training_stats = []

# time measure
total_t0 = time.time()


for epoch_i in range(0, epochs):
    # ========================================
    # Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    #training time for epoch
    t0 = time.time()

    # reset total loss
    total_train_loss = 0

    # change model mode to training
    model.train()

    for step, batch in enumerate(train_dataloader):

        # Progress update every 20 batches.
        if step % 40 == 0 and not step == 0:
            # elapsed time
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        #clear calculated gradiends
        model.zero_grad()        

        # train model with current batch
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # training loss of all batches for statistics
        total_train_loss += loss.item()

        # calculate gradients
        #loss.backward() ####changed for apex fp16
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update the learning rate
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
        
    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")

    t0 = time.time()
    # put the model in evaluation mode
    model.eval()

    # tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # constructing the compute graph is only needed for training
        with torch.no_grad():        

           
            # the documentation for this model function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # accumulate the validation loss.
        total_eval_loss += loss.item()

        # move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # calculate the accuracy 
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


## 5.7 Einzelinferenz
Hier können eigene Sätze mit dem Modell getestet werden

In [ ]:
to_predict="@DB_Bahn heute wieder später"

def fromOrdinal(n):
    if n==2:
        return 'positiv'
    if n==1:
        return 'neutral'
    if n==0:
        return 'negativ'

#preprocess input
df=pd.DataFrame([to_predict])
df=df.apply(data_cleaner.clean_text)
to_predict=(df[0].item())

print("Vorverarbeiteter Text: '",to_predict,"'")

tokenized=tokenizer.encode(to_predict[:512])
print("\nTokenized: ",tokenized)


#predict input
input_ids = torch.tensor([tokenized]).cuda()
with torch.no_grad(): 
    model.eval()
output=model(input_ids,token_type_ids=None)

#parse output
labels=[]
for tensor in output[0][0]:
    labels.append(tensor.item())

predicted=fromOrdinal(np.argmax(labels))
print("\nErgebnis: ",predicted)

In [ ]:
 torch.cuda.empty_cache()